In [1]:
# zipurl = 'https://www2.census.gov/programs-surveys/acs/data/pums/2016/5-Year/csv_pus.zip'
zipurl = 'https://www2.census.gov/programs-surveys/acs/data/pums/2015/1-Year/csv_pus.zip'
output_path = './csv_pus/'

download = False

if download:
    from io import BytesIO
    from urllib.request import urlopen
    from zipfile import ZipFile 
    with urlopen(zipurl) as zipresp:
        with ZipFile(BytesIO(zipresp.read())) as zfile:
            zfile.extractall('output_path')

In [2]:
from tqdm import tqdm

if download:
  import requests
  response = requests.get(zipurl, stream=True)
  with open("csv_pus/csv_pus.zip", "wb") as handle:
    num_bytes = 0
    for data in (pbar := tqdm(response.iter_content(chunk_size=1024))):
        num_bytes += len(data)
        pbar.set_description(f"Downloading csv_pus.zip ... {num_bytes/(1024**2):.2f} MB")
        handle.write(data)


Trick specific for some versions of cuda/jaxlib.

In [1]:
%env XLA_FLAGS=--xla_gpu_force_compilation_parallelism=1

env: XLA_FLAGS=--xla_gpu_force_compilation_parallelism=1


In [4]:
import jax
import jax.numpy as jnp
import numpy as onp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [5]:
from sklearn.metrics.pairwise import PAIRWISE_KERNEL_FUNCTIONS
PAIRWISE_KERNEL_FUNCTIONS

{'additive_chi2': <function sklearn.metrics.pairwise.additive_chi2_kernel(X, Y=None)>,
 'chi2': <function sklearn.metrics.pairwise.chi2_kernel(X, Y=None, gamma=1.0)>,
 'linear': <function sklearn.metrics.pairwise.linear_kernel(X, Y=None, dense_output=True)>,
 'polynomial': <function sklearn.metrics.pairwise.polynomial_kernel(X, Y=None, degree=3, gamma=None, coef0=1)>,
 'poly': <function sklearn.metrics.pairwise.polynomial_kernel(X, Y=None, degree=3, gamma=None, coef0=1)>,
 'rbf': <function sklearn.metrics.pairwise.rbf_kernel(X, Y=None, gamma=None)>,
 'laplacian': <function sklearn.metrics.pairwise.laplacian_kernel(X, Y=None, gamma=None)>,
 'sigmoid': <function sklearn.metrics.pairwise.sigmoid_kernel(X, Y=None, gamma=None, coef0=1)>,
 'cosine': <function sklearn.metrics.pairwise.cosine_similarity(X, Y=None, dense_output=True)>}

* 50 states
* 979 regions
* averaging to 19.58 regions per state
* Over 2 400 000 individuals where used for the analysis
* The average number of individuals per state is 48 000
* The average number of individuals per region is 2 500

In [6]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import explained_variance_score


def fit_ridge(x_train, x_test, y_train, y_test, lbda, gamma, *, log_prob=False, algo='ridge'):
  if log_prob:
    y_train = onp.log(y_train)

  if gamma == 'scale':
    gamma = 1 / (x_train.shape[1] * x_train.var())
  elif gamma == 'auto':
    gamma = 1 / x_train.shape[1]

  if algo == 'ridge':
    kernel = 'linear'
    x_train = x_train / gamma**0.5
  elif algo == 'rbf':
    kernel = 'rbf'
  
  if algo == 'baseline':
    y_pred = y_train.mean() * onp.ones_like(y_pred)
  else:
    model = KernelRidge(kernel=kernel,
                        alpha=lbda,
                        gamma=gamma)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
  if log_prob:
    y_pred = onp.exp(y_pred)
    y_pred = onp.clip(y_pred, 0, 1)

  evs = explained_variance_score(y_test, y_pred)
  l2 = ((y_test - y_pred)**2).mean()
  mae = (onp.abs(y_test - y_pred)).mean()
  corr = onp.corrcoef(y_test, y_pred)[0, 1]

  return model, (evs*100, l2**0.5*100, mae*100, corr*100, y_pred.mean()*100)

In [7]:
def load_preprocess_embeddings(filename, *, verbose=False):
  npz_file = onp.load(filename, allow_pickle=True)

  if verbose:
    for item in npz_file.files:
      print(item, npz_file[item].shape)
  
  df = pd.DataFrame(npz_file['emb_lin'])

  if verbose:
    print(df.describe())

  if 'rbf' in filename:
    embeddings = npz_file['emb_rff']
  else:
    embeddings = npz_file['emb_lin']
  
  print(f"Embeddings: {embeddings.shape}")

  model = StandardScaler(copy=True, with_mean=True, with_std=True)
  embeddings = model.fit_transform(embeddings)

  labels = pd.read_csv('results-2016-election.csv')
  if verbose:
    print(labels.describe())

  regions = npz_file['region_names']
  if verbose:
    print("Regions names:", regions)

  assert all(regions == labels['region']), "Align regions and labels first"

  total = labels['votes_R'] + labels['votes_D'] + labels['votes_oth']
  labels['perc_R'] = labels['votes_R'] / total
  labels['perc_D'] = labels['votes_D'] / total
  labels['perc_oth'] = labels['votes_oth'] / total

  if verbose:
    print(labels.describe())

  return embeddings, labels, regions

In [8]:
def load_fit(filename, category, random_state, log_prob, *, algo='ridge'):
  embeddings, labels, regions = load_preprocess_embeddings(filename)
  labels = labels[category].values
  splits = train_test_split(embeddings, labels, test_size=0.2, random_state=random_state)
  model, metrics = fit_ridge(*splits, lbda=1e-3, gamma='scale', log_prob=log_prob, algo=algo)
  return model, metrics

In [13]:
import time
results_df, i = pd.DataFrame(columns=['cat', 'log_prob', 'random_state', 'evs', 'l2', 'mae', 'corr', 'y_pred', 'time']), 0
states = [42, 43, 44, 45, 46]
algo = 'rbf'
filename = 'embeddings/embeddings_swd_32_32.npz'
for random_state in states:  # reproducible 5-fold validation.
  for log_prob in [False]:  
    for cat in ['perc_R', 'perc_D']: 
      start = float(time.time())
      model, metrics = load_fit(filename=filename, category=cat, random_state=random_state, log_prob=log_prob, algo=algo)
      evs, l2, mae, corr, y_pred = metrics
      # print(f"[{cat},log_scale={log_scale}] evs={evs:.2f} l2={l2:.2f} mae={mae:.2f} corr={corr:.2f}")
      end = float(time.time())
      delta = end - start
      results_df.loc[i] = [cat, log_prob, random_state, evs, l2, mae, corr, y_pred, delta]
      i += 1
results_df

Embeddings: (979, 1024)
Embeddings: (979, 1024)
Embeddings: (979, 1024)
Embeddings: (979, 1024)
Embeddings: (979, 1024)
Embeddings: (979, 1024)
Embeddings: (979, 1024)
Embeddings: (979, 1024)
Embeddings: (979, 1024)
Embeddings: (979, 1024)


,cat,log_prob,random_state,evs,l2,mae,corr,y_pred,time
0,perc_R,False,42,67.106345,9.228085,6.752106,82.098390,54.445619,0.151901
1,perc_D,False,42,74.767677,7.773080,5.897993,86.563462,38.030340,0.212781
2,perc_R,False,43,73.745561,7.981404,6.153551,86.236474,56.066589,0.142646
3,perc_D,False,43,75.781198,7.348973,5.637033,87.052885,36.089425,0.129882
4,perc_R,False,44,71.688589,8.044618,5.847027,84.977603,53.442216,0.144054
5,perc_D,False,44,70.984058,8.223803,5.804627,84.338742,38.340345,0.156052
6,perc_R,False,45,78.607038,7.557678,5.866358,89.211967,54.328600,0.183404
7,perc_D,False,45,62.157756,9.944106,6.934715,78.847402,36.212643,0.159550
8,perc_R,False,46,72.558707,8.527476,6.446398,85.632604,53.835909,0.162898
9,perc_D,False,46,67.496721,8.991726,6.335662,82.156477,38.116250,0.163058


In [14]:
grouped = results_df.groupby(['cat', 'log_prob']).agg(["mean", "std"]).reset_index()
grouped

cat log_prob random_state                  evs                  l2  \
                           mean       std       mean       std      mean   
0  perc_D    False         44.0  1.581139  70.237482  5.581366  8.456338   
1  perc_R    False         44.0  1.581139  72.741248  4.132484  8.267852   

                  mae                 corr               y_pred            \
        std      mean       std       mean       std       mean       std   
0  1.030444  6.122006  0.522555  83.791794  3.381467  37.357801  1.108277   
1  0.637512  6.213088  0.388182  85.631408  2.554221  54.423787  1.002175   

       time            
       mean       std  
0  0.164265  0.030113  
1  0.156981  0.016819

In [15]:
grouped['time']['mean'].sum(), (grouped['time']['std']**2).sum()**0.5

(0.32124538421630855, 0.03449180540354996)

In [13]:
embeddings, labels, regions = load_preprocess_embeddings('embeddings_rbf.npz')

Embeddings: (979, 4096)


In [14]:
labels

,region,votes_R,votes_oth,votes_D,perc_R,perc_D,perc_oth
0,AK_10_01,129786,22947,92013,0.530289,0.375953,0.093758
1,AL_10_01,69431,2750,17463,0.774519,0.194804,0.030677
2,AL_10_02,24855,914,30231,0.443839,0.539839,0.016321
3,AL_10_03,24735,562,37647,0.392968,0.598103,0.008929
4,AL_10_04,54387,1234,7700,0.858909,0.121603,0.019488
...,...,...,...,...,...,...,...
974,WY_10_01,32445,5460,18460,0.575623,0.327508,0.096869
975,WY_10_02,36147,3854,14473,0.663564,0.265686,0.070749
976,WY_10_03,37382,2370,4817,0.838744,0.108080,0.053176
977,WY_10_04,33452,3273,8520,0.739352,0.188308,0.072339


In [15]:
y_pred = dict()

splits = train_test_split(embeddings, labels, test_size=0.5, random_state=random_state)
for perc in ['perc_R', 'perc_D']:
  xtrain, xtest, ytrain, ytest = splits
  model, metrics = fit_ridge(xtrain, xtest, ytrain[perc], ytest[perc], lbda=1e-3, gamma='scale', algo='rbf')
  print(metrics)
  ytest_realigned_1 = pd.Series(model.predict(xtest))
  # use same index as xtest
  ytest_realigned_1.index = ytest['region']
  # print(ytest_realigned_1)
  xtest, xtrain, ytest, ytrain = xtrain, xtest, ytrain, ytest  # swap splits for 2-fold validation.
  model, metrics = fit_ridge(xtrain, xtest, ytrain[perc], ytest[perc], lbda=1e-3, gamma='scale', algo='rbf')
  print(metrics)
  ytest_realigned_2 = pd.Series(model.predict(xtest))
  # use same index as xtest
  ytest_realigned_2.index = ytest['region']
  # print(ytest_realigned_2)
  ytest_realigned = pd.concat([ytest_realigned_1, ytest_realigned_2], axis=0)
  # sort rows by index
  ytest_realigned = ytest_realigned.sort_index()
  y_pred[perc] = ytest_realigned
y_pred = pd.DataFrame(y_pred)
y_pred


(76.5276860166276, 7.58737474144042, 5.8206076814940975, 88.30141313053123, 55.01561420560025)
(76.79902231851253, 7.514593652010361, 5.633089632985508, 88.0917097672175, 55.28084896492999)
(70.83328481938061, 8.360145899482163, 5.2216524938169835, 84.16260070331218, 37.27440159732984)
(73.36321316417286, 7.944637005206475, 4.975546649954138, 85.65233591830406, 36.13645446385242)


,perc_R,perc_D
region,,
AK_10_01,0.467791,0.456297
AL_10_01,0.713251,0.258021
AL_10_02,0.491651,0.633228
AL_10_03,0.338806,0.418264
AL_10_04,0.714448,0.146228
...,...,...
WY_10_01,0.639050,0.329783
WY_10_02,0.567776,0.260660
WY_10_03,0.613931,0.202910


In [16]:
y_pred.describe()

,perc_R,perc_D
count,979.000000,979.000000
mean,0.551481,0.367060
std,0.151194,0.128197
min,0.058762,0.111528
25%,0.459590,0.269719
50%,0.563847,0.345336
75%,0.665854,0.451052
max,0.924509,0.875237


In [17]:
y_pred.to_csv('y_pred.csv')